In [15]:
using DataFrames
using Recommender: Movielens100k, load_dataset, rows2sparse, leave_one_out_split, reindex_id_column!

In [6]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [21]:
df = DataFrame(rating)
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,196,242,3,881250949
2,186,302,3,891717742
3,22,377,1,878887116
4,244,51,2,880606923
5,166,346,1,886397596
6,298,474,4,884182806
7,115,265,2,881171488
8,253,465,5,891628467
9,305,451,3,886324817


In [28]:
df[!, :rating] .= 1
first(df, 10)

,userid,movieid,rating,timestamp,data_split
,Int64,Int64,Int64,Int64,Symbol
1,196,110,1,881252305,test
2,196,94,1,881252172,valid
3,196,1118,1,881252128,train
4,196,108,1,881252110,train
5,196,411,1,881252090,train
6,196,580,1,881252056,train
7,196,67,1,881252017,train
8,196,692,1,881252017,train
9,196,25,1,881251955,train


In [29]:
df = leave_one_out_split(df, col_group = :userid, col_sort = :timestamp);

In [30]:
df_train = df[df.data_split.==:train, :]
df_valid = df[df.data_split.==:valid, :]
df_test = df[df.data_split.==:test, :]
first(df_train, 10)

,userid,movieid,rating,timestamp,data_split
,Int64,Int64,Int64,Int64,Symbol
1,196,1118,1,881252128,train
2,196,108,1,881252110,train
3,196,411,1,881252090,train
4,196,580,1,881252056,train
5,196,67,1,881252017,train
6,196,692,1,881252017,train
7,196,25,1,881251955,train
8,196,13,1,881251955,train
9,196,762,1,881251955,train


In [31]:
df_train, user2uidx = reindex_id_column!(df_train, :userid)
df_train, item2iidx = reindex_id_column!(df_train, :movieid);

In [32]:
first(df_train, 10)

,userid,movieid,rating,timestamp,data_split,userid_index,movieid_index
,Int64,Int64,Int64,Int64,Symbol,Int64,Int64
1,196,1118,1,881252128,train,1,1
2,196,108,1,881252110,train,1,2
3,196,411,1,881252090,train,1,3
4,196,580,1,881252056,train,1,4
5,196,67,1,881252017,train,1,5
6,196,692,1,881252017,train,1,6
7,196,25,1,881251955,train,1,7
8,196,13,1,881251955,train,1,8
9,196,762,1,881251955,train,1,9


In [ ]:
X_valid = 

In [35]:
X_train = rows2sparse(df_train, col_user=:userid_index, col_item=:movieid_index, col_rating=:rating);

In [38]:
using Recommender: tfidf, bm25, compute_similarity, predict_u2i

In [37]:
function train_item_knn(X, k, shirnk, weighting, normalize)
    if weighting==:tfidf
        X = tfidf(X)
    elseif weighting==:bm25
        X=bm25(X)
    end

    return compute_similarity(X, k, shirnk, normalize)
end

train_item_knn (generic function with 1 method)

In [ ]:
function predict_u2i(similarity, user_histories, userids, user2uidx, item2iidx, n)
    uidxs = [user2uidx[userid] for userid in userids]
    preds = [predict_u2i(similarity, user_histories[uidx, :], n) for uidxs]
end

In [ ]:
function evaluate_item_knn(X_train, X_valid, k, shirnk, weighting, normalize)
    similarity = train_item_knn(X_train, k, shirnk, weighting, normalize)